# **Исследование надёжности заёмщиков**

Исследование необходимо для кредитного подразделения банка. Задача - влияет ли семейное положение и количество детей клиента на факт погашения кредита в срок. Входные данные от банка — статистика о платёжеспособности клиентов.

Результаты исследования необходимы при построении модели **кредитного скоринга**

### Описание данных

children — количество детей в семье

days_employed — общий трудовой стаж в днях

dob_years — возраст клиента в годах

education — уровень образования клиента

education_id — идентификатор уровня образования

family_status — семейное положение

family_status_id — идентификатор семейного положения

gender — пол клиента

income_type — тип занятости

debt — имел ли задолженность по возврату кредитов

total_income — ежемесячный доход

purpose — цель получения кредита

## Шаг 1. Импорт библиотек и загрузка файла с данными

### Загрузка и оценка файла данных

In [2]:
# Импорт библиотек, загрузка файла данных
import pandas as pd
import numpy as np
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)
try:
    data = pd.read_csv('/datasets/data.csv')
except:
    data = pd.read_csv('data.csv')

In [3]:
# Вызов общей информации о данных, визуальная оценка
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21525 entries, 0 to 21524
Data columns (total 12 columns):
children            21525 non-null int64
days_employed       19351 non-null float64
dob_years           21525 non-null int64
education           21525 non-null object
education_id        21525 non-null int64
family_status       21525 non-null object
family_status_id    21525 non-null int64
gender              21525 non-null object
income_type         21525 non-null object
debt                21525 non-null int64
total_income        19351 non-null float64
purpose             21525 non-null object
dtypes: float64(2), int64(5), object(5)
memory usage: 2.0+ MB


<span style='color:blue'>В Dataframe присутствуют **пропуски** в столбце **days_employed** в **2174** строках и в столбце **total_income** в **2174** строках. Тип данных в столбцах соответствует логике названия столбцов</span>

In [4]:
#Проверка наличия пробелов в названии столбцов
data.columns

Index(['children', 'days_employed', 'dob_years', 'education', 'education_id',
       'family_status', 'family_status_id', 'gender', 'income_type', 'debt',
       'total_income', 'purpose'],
      dtype='object')

<span style='color:blue'>В названиях столбцов отсутствуют лишние пробелы</span>

### Визуальная оценка сэмплов данных

In [5]:
# Вывод первых пяти строк Dataframe для визуальной оценки
data.head()

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
0,1,-8437.673028,42,высшее,0,женат / замужем,0,F,сотрудник,0,253875.639453,покупка жилья
1,1,-4024.803754,36,среднее,1,женат / замужем,0,F,сотрудник,0,112080.014102,приобретение автомобиля
2,0,-5623.422610,33,Среднее,1,женат / замужем,0,M,сотрудник,0,145885.952297,покупка жилья
3,3,-4124.747207,32,среднее,1,женат / замужем,0,M,сотрудник,0,267628.550329,дополнительное образование
4,0,340266.072047,53,среднее,1,гражданский брак,1,F,пенсионер,0,158616.077870,сыграть свадьбу


In [6]:
# Вывод последних пяти строк Dataframe для визуальной оценки
data.tail()

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
21520,1,-4529.316663,43,среднее,1,гражданский брак,1,F,компаньон,0,224791.862382,операции с жильем
21521,0,343937.404131,67,среднее,1,женат / замужем,0,F,пенсионер,0,155999.806512,сделка с автомобилем
21522,1,-2113.346888,38,среднее,1,гражданский брак,1,M,сотрудник,1,89672.561153,недвижимость
21523,3,-3112.481705,38,среднее,1,женат / замужем,0,M,сотрудник,1,244093.050500,на покупку своего автомобиля
21524,2,-1984.507589,40,среднее,1,женат / замужем,0,F,сотрудник,0,82047.418899,на покупку автомобиля


<span style='color:blue'>В сэмплах строк столбца days_employed заметны **минусовые значения** и **аномально высокие плюсовые** (**343937 дней стажа**) значения для логики столбца.</span>

In [7]:
# Перевод сэмплов строки с аномально высоким плюсовым значением стажа в днях в годы
print('Cтаж в годах по этой строке {:.0f}'.format(data.loc[4,'days_employed'] / 365))

Cтаж в годах по этой строке 932


In [8]:
print('Cтаж в годах по этой строке {:.0f}'.format(data.loc[21521,'days_employed'] / 365))

Cтаж в годах по этой строке 942


<span style='color:blue'>Полученные значения в двух сэмплах выше вероятно **ошибочные**, у человека такой стаж **невозможен** даже с применением коэффицентов для льготных категорий</span>.
Возможно аномально большие числовые значения со знаком плюс, а также знак минус в остальных значениях - результат вычисления определенного алгоритма, или кода с ошибкой когда они попадают в базу, либо при выгрузке, либо компиляция из разных баз.

<div class="alert alert-warning" style= "background-color: #d3e5f0" style="border-radius: 15px">Возможно аномально большие числовые значения со знаком плюс, а также знак минус в остальных значениях - результат вычисления определенного алгоритма, или кода с ошибкой когда они попадают в базу, либо при выгрузке, либо компиляция из разных баз.
</div>

#### Вывод описания колличественных признаков

In [9]:
# Вывод описания колличественных признаков с форматированием вывода через переменную categorial_data_summary и метод describe()
categorial_data_summary = data.describe()
categorial_data_summary.loc['count'] = categorial_data_summary.loc['count'].astype(int).astype(str)
categorial_data_summary.iloc[1:] = categorial_data_summary.iloc[1:].applymap('{:.2f}'.format)
categorial_data_summary

,children,days_employed,dob_years,education_id,family_status_id,debt,total_income
count,21525,19351,21525,21525,21525,21525,19351
mean,0.54,63046.50,43.29,0.82,0.97,0.08,167422.30
std,1.38,140827.31,12.57,0.55,1.42,0.27,102971.57
min,-1.00,-18388.95,0.00,0.00,0.00,0.00,20667.26
25%,0.00,-2747.42,33.00,1.00,0.00,0.00,103053.15
50%,0.00,-1203.37,42.00,1.00,0.00,0.00,145017.94
75%,1.00,-291.10,53.00,1.00,1.00,0.00,203435.07
max,20.00,401755.40,75.00,4.00,4.00,1.00,2265604.03


<span style='color:blue'><br>В столбце **сhildren** замечен артефакт **минус 1 ребенок**, и вероятная ошибка **20 детей**. 
<br>В **столбце days_employed** подтверждается вывод об ошибочности положительных данных о стаже max значение стажа в годах = **1100 лет**. 
<br>В столбце **dob_years** замечена явная ошибка - **возраст 0 лет** как **min** значение.</span>

#### Вывод описания категориальных признаков

In [10]:
#Вывод описания категориальных признаков через метод describe()
data.describe(include=[object])

,education,family_status,gender,income_type,purpose
count,21525,21525,21525,21525,21525
unique,15,5,3,8,38
top,среднее,женат / замужем,F,сотрудник,свадьба
freq,13750,12380,14236,11119,797


<br><span style='color:blue'>Замечено ошибочное значение в столбце **gender** - **третье значение для признака пола**</span>
<br><span style='color:blue'> Получен *предварительный* мини-портрет заемщика: женщина, имеет среднее образование, замужем, работает по найму, кредит берет на свадьбу.</span>

In [11]:
data['gender'].unique() # Вывод всех значени пола для контроля

array(['F', 'M', 'XNA'], dtype=object)

<span style='color:blue'>Ошибочное значение признака пола - **'XNA'**</span>

**Вывод**

<br>Dataframe содержит 21525 строк в 12 столбцах, типы данных в них соответствуют логике. 
<br><span style='color:darkblue'>**Пропуски**</span> данных в столбцах **days_employed** и **total_income** в количестве <span style='color:darkblue'>**2174**</span> строк.
<br>В столбце **days_employed** присутствуют <span style='color:darkblue'>**минусовые и положительные значения**</span>, среди положительных значений присутствуют <span style='color:darkblue'>**ошибки данных**</span>, например в **строке 4** - стаж в днях **340266.072047** в годах - <span style='color:darkblue'>**932 года**</span>. 
<br>В столбце **children** выявленна явная <span style='color:darkblue'>**ошибка**</span> данных - <span style='color:darkblue'>**минусовое значение**</span>для количества детей и аномально высокое <span style='color:darkblue'>**max значение - 20 детей**</span>. 
<br>В столбце **dob_years** выявлена явная <span style='color:darkblue'>**ошибка**</span> данных - <span style='color:darkblue'>**нулевое значения для возраста**</span>. 
<br>В столбце **gender** выявлен <span style='color:darkblue'>**ошибочный признак**</span> пола - значение <span style='color:darkblue'>**XNA**.</span></span>

## Шаг 2. Предобработка данных

### Обработка пропусков

#### Проверка взаимосвязи пропусков в столбцах days_employed и total_income по строкам

In [12]:
# Процент пропусков по столбцам в Dataframe
empty_spaces = data.copy()
for col in empty_spaces.columns:
    str_missing = np.mean(empty_spaces[col].isnull())
    print('{} - {}%'.format(col, round(str_missing*100)))

children - 0%
days_employed - 10%
dob_years - 0%
education - 0%
education_id - 0%
family_status - 0%
family_status_id - 0%
gender - 0%
income_type - 0%
debt - 0%
total_income - 10%
purpose - 0%


In [13]:
# Проверка предположения - в каждой строке с пропуском в столбце days_employed есть пропуск в столбце income_type
null_data = empty_spaces[empty_spaces.isnull().any(axis=1)]
null_data.describe()

,children,days_employed,dob_years,education_id,family_status_id,debt,total_income
count,2174.000000,0.0,2174.000000,2174.000000,2174.000000,2174.000000,0.0
mean,0.552438,NaN,43.632015,0.800828,0.975161,0.078197,NaN
std,1.469356,NaN,12.531481,0.530157,1.418220,0.268543,NaN
min,-1.000000,NaN,0.000000,0.000000,0.000000,0.000000,NaN
25%,0.000000,NaN,34.000000,0.250000,0.000000,0.000000,NaN
50%,0.000000,NaN,43.000000,1.000000,0.000000,0.000000,NaN
75%,1.000000,NaN,54.000000,1.000000,1.000000,0.000000,NaN
max,20.000000,NaN,73.000000,3.000000,4.000000,1.000000,NaN


In [14]:
# Количество дебиторов в строках где пропущены значения days_employed и total_income
null_data = empty_spaces.fillna(0)
debitors_in_null_data = null_data.loc[(null_data['days_employed'] == 0) & (null_data['total_income'] == 0)]
debitors_in_null_data['debt'].sum()

170

In [15]:
# Количество дебиторов в общем Dataframe
debitors_in_data = empty_spaces['debt'].sum()
debitors_in_data

1741

In [16]:
# Группировка пропущенных значений стажа по типу дохода
null_data = null_data.loc[null_data['days_employed'] == 0]
null_data.groupby('income_type')['days_employed'].count()

income_type
госслужащий         147
компаньон           508
пенсионер           413
предприниматель       1
сотрудник          1105
Name: days_employed, dtype: int64

<br><span style='color:blue'>Количество **должников** в строках с пропусками - 10% что соотносится с количеством строк с пропусками в общем Dataframe - 10%, что означает что в пропущеных строках такое же распределения должников, как и в общем Dataframe</span>
<br><span style='color:blue'>Если в строке есть **пропуск** в стаже по столбцу **days_employed** то данные о доходе в столбце **total_income** также **отсутствуют**, возможно такие заемщики не могут потвердить стаж и доход официально</span>

#### Замена пропусков в столбце стажа days_employed

In [17]:
# Оценка значений стажа в столбце days_employed со знаком плюс в годах
data_in_years = data.copy()
data_in_years['days_employed']= data_in_years['days_employed'].apply(lambda x: x/365)
positve_days_employed = data_in_years[data['days_employed'] > 0]
positve_days_employed.describe().applymap('{:.2f}'.format)

,children,days_employed,dob_years,education_id,family_status_id,debt,total_income
count,3445.00,3445.00,3445.00,3445.00,3445.00,3445.00,3445.00
mean,0.13,1000.01,59.12,0.91,0.98,0.05,137124.11
std,0.96,57.74,7.58,0.52,1.32,0.22,80242.21
min,-1.00,900.63,0.00,0.00,0.00,0.00,20667.26
25%,0.00,949.70,56.00,1.00,0.00,0.00,82876.34
50%,0.00,1000.58,60.00,1.00,0.00,0.00,118514.49
75%,0.00,1049.99,64.00,1.00,2.00,0.00,169746.26
max,20.00,1100.70,74.00,4.00,4.00,1.00,735103.27


In [18]:
negative_days_employed = data[(data['days_employed'] < 0) & (data['days_employed'] == 'пенсионер')]
negative_days_employed

/opt/conda/lib/python3.7/site-packages/pandas/core/ops/__init__.py:1115: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  result = method(y)


,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose


<br><span style='color:blue'>Как можно видеть позитивные значения в столбце стажа days_employed в 3445 строках. Все они - аномальные, если перевести дни в годы, **min** значение стажа в этих строках - **900 лет**, среднее - **1000 лет** </span>

In [19]:
# Еще раз вывод состава столбца income_type - тип дохода
data['income_type'].value_counts()

сотрудник          11119
компаньон           5085
пенсионер           3856
госслужащий         1459
безработный            2
предприниматель        2
студент                1
в декрете              1
Name: income_type, dtype: int64

In [20]:
# Вывод инфо кто имеет аномальные плюсовые значения стажа, т.е. запредельный стаж
positve_days_employed.groupby('income_type')['days_employed'].count()

income_type
безработный       2
пенсионер      3443
Name: days_employed, dtype: int64

In [21]:
# Проверка - есть ли пенсионеры в строках с отрицательным стажем, который логичен по значениям
negative_days_employed = data[(data['days_employed'] < 0) & (data['days_employed'] == 'пенсионер')]
negative_days_employed.groupby('income_type')['days_employed'].count()

Series([], Name: days_employed, dtype: int64)

<br><span style='color:blue'> Все пенсионеры имеют либо пропуск (413 строк) в стаже либо аномально высокие плюсовые значения (3443 строки). У обоих безработных из выборки - тоже аномально высокий плюсовой стаж.  </span>

In [22]:
# Заполнение пропусков в столбце days_employed медианой по группам nbgf lj[jlf b djphfcnf]
med = data.groupby(['income_type','dob_years'])['days_employed'].transform('median')
data['days_employed'].fillna(med,inplace=True)

In [23]:

print('Количество пропусков:',data['days_employed'].isnull().sum()) # Проверка

Количество пропусков: 4


<div class="alert alert-warning" style= "background-color: #d3e5f0" style="border-radius: 15px"><br><span style='color:blue'>После подтерждения гипотезы что аномально высокий плюсовой стаж только у пенсионеров и безработных, пропуски в столбце стажа days_employed заполнены медианой по столбцам типа дохода и возраста.
<br>Для выяснения причин появляния аномального стажа в виде плюсовых значений и логичного стажа - в виде минусовых, необходимо сделать запрос к поставщику данных об источнике и порядке их формирования.</span></div>

#### Замена минусового значения в количестве детей

In [1116]:
# Устранение минусового значения в 47 строках
data['children'] = data['children'].abs()
# Проверка
data[data['children'] < 0]

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose


In [1117]:
print("Количество заемщиков числом детей 20 -", data[data['children'] == 20].count()[0])

Количество заемщиков числом детей 20 - 76


In [1118]:
# Приведение количества детей от 20 к 2
data.loc[data['children'] == 20, 'children'] = 2
data.loc[data['children'] == 20] # Проверка 

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose


<br><span style='color:blue'>47 строк (**0,21%** от общего количества) со значением **-1** были приведены к значению **1**, исходя из логики ошибочности минуса при заполнении данных. Необходимо также уточнить у поставщика данных причны появления таких записей, хотя в процентном соотношении они ничтожны для объема данных.</span>
<br><span style='color:blue'> Количество детей - **20**, исходя из описания колличественных столбцов проведенного ранее, среднее количество детей в данных - 0,54, среднее отклонение - 1.38, поэтому можно считать **76** (**0.35%** от общего количества) строк с количеством детей - 20 явной ошибкой заполнения данных. Эти строке приведены **к значению 2 ребенка**. Это не приведет к искажению данных в виду незначительности доли таких значений. Необходимо также сделать запрос поставщику данных о причинах возникновения таких аномальных значений, если существуют семьи с таким количеством детей то их вероятность попасть представленную выборку очень маловерятна.</span>

#### Замена нулевых значений возраста в столбце возраста dob_years

In [1119]:
# Оценка количества значений 0 в столбце dob_years
print('Количество нулевых значений в столбце dob_years:',data['dob_years'][data['dob_years'] == 0].count())
print(
    '% нулевых значений в столбце dob_years:',(len(data['dob_years'].value_counts(normalize=True)) / len(
        data['dob_years']))*100)

Количество нулевых значений в столбце dob_years: 101
% нулевых значений в столбце dob_years: 0.2694541231126597


In [1120]:
# Оценка типов занятости income_type для строк с нулевым возрастом
data.loc[data['dob_years'] == 0, 'income_type'].value_counts()

сотрудник      55
пенсионер      20
компаньон      20
госслужащий     6
Name: income_type, dtype: int64

In [1121]:
# Замена значений нулевого возраста в основном Dataframe на медианный возраст соответствующий типу занятости по столбцу income_type
median_age = data.groupby('income_type')['dob_years'].median()
data.loc[(data['dob_years'] == 0) & (data['income_type'] == 'сотрудник'), 'dob_years'] = median_age[6]
data.loc[(data['dob_years'] == 0) & (data['income_type'] == 'пенсионер'), 'dob_years'] = median_age[4]
data.loc[(data['dob_years'] == 0) & (data['income_type'] == 'компаньон'), 'dob_years'] = median_age[3]
data.loc[(data['dob_years'] == 0) & (data['income_type'] == 'госслужащий'), 'dob_years'] = median_age[2]
data[data['dob_years'] == 0]

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose


<br><span style='color:blue'>Нулевые значения возраста заменены на **медианные**, для соответствующего типа занятости **income_type** т.к. % таких значений незначителен и составляет **0.26**</span>

#### Замена ошибочного значения пола XNA в столбце gender

In [1122]:
# Поиск строки с ошибочным значением, оценка количества
data[data['gender'] == 'XNA']

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
10701,0,-2358.60,24.00,неоконченное высшее,2,гражданский брак,1,XNA,компаньон,0,203905.16,покупка недвижимости


In [1123]:
# Замена значения XMA
data.loc[10701, 'gender'] = data.loc[10701, 'gender'].replace('XNA','F')
data[data['gender'] == 'XNA'] # Проверка

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose


<br><span style='color:blue'>Ошибочное значение **XMA** в столбце **gender** заменено на **F**, Т.к. ему соответствует **family_status_id = 1**</span>

#### Замена пропусков дохода в столбцe total_income

In [1124]:
# Просмотр min/max значений, среднего и медианы дохода по типам занятости incom_type
pd.set_option('display.float_format', '{:.2f}'.format)
data.groupby(
    'income_type').agg(min_income=("total_income", "min"), max_income=pd.NamedAgg(column = "total_income", aggfunc="max"),
mean_income=("total_income", np.mean),median_income=pd.NamedAgg(column = "total_income", aggfunc='median'))

,min_income,max_income,mean_income,median_income
income_type,,,,
безработный,59956.99,202722.51,131339.75,131339.75
в декрете,53829.13,53829.13,53829.13,53829.13
госслужащий,29200.08,910451.47,170898.31,150447.94
компаньон,28702.81,2265604.03,202417.46,172357.95
пенсионер,20667.26,735103.27,137127.47,118514.49
предприниматель,499163.14,499163.14,499163.14,499163.14
сотрудник,21367.65,1726276.01,161380.26,142594.40
студент,98201.63,98201.63,98201.63,98201.63


In [1125]:
# оценка среднего и квартилей дохода до заполнения пропусков
data.describe()

,children,days_employed,dob_years,education_id,family_status_id,debt,total_income
count,21525.00,21521.00,21525.00,21525.00,21525.00,21525.00,19351.00
mean,0.48,63509.94,43.50,0.82,0.97,0.08,167422.30
std,0.76,141126.94,12.23,0.55,1.42,0.27,102971.57
min,0.00,-18388.95,19.00,0.00,0.00,0.00,20667.26
25%,0.00,-2573.91,34.00,1.00,0.00,0.00,103053.15
50%,0.00,-1283.69,43.00,1.00,0.00,0.00,145017.94
75%,1.00,-316.24,53.00,1.00,1.00,0.00,203435.07
max,5.00,401755.40,75.00,4.00,4.00,1.00,2265604.03


In [1126]:
# Замена пропусков в доходе по медиане соотествующего значения типа занятости и образования
med = data.groupby(['income_type','education'])['total_income'].transform('median')
data['total_income'].fillna(med,inplace=True)
print('Количество пропусков в total_income:',data['total_income'].isnull().sum()) # Проверка

Количество пропусков в total_income: 0


In [1127]:
# Оценка изменений в средней и квартилях дохода после заполнения пропусков
data.describe()

,children,days_employed,dob_years,education_id,family_status_id,debt,total_income
count,21525.00,21521.00,21525.00,21525.00,21525.00,21525.00,21525.00
mean,0.48,63509.94,43.50,0.82,0.97,0.08,165375.89
std,0.76,141126.94,12.23,0.55,1.42,0.27,98164.25
min,0.00,-18388.95,19.00,0.00,0.00,0.00,20667.26
25%,0.00,-2573.91,34.00,1.00,0.00,0.00,107714.24
50%,0.00,-1283.69,43.00,1.00,0.00,0.00,143111.63
75%,1.00,-316.24,53.00,1.00,1.00,0.00,198105.57
max,5.00,401755.40,75.00,4.00,4.00,1.00,2265604.03


<br><span style='color:blue'>Для **замены пропусков** дохода в **total_income** выбраны **медианные значения** соответствующего типа занятости **income_type** т.к. в столбце income_type все значения даны в полном объеме и логически связаны с уровнем дохода</span>

<div class="alert alert-warning" style= "background-color: #d3e5f0" style="border-radius: 15px"><span style='color:blue'>Выполнена новая группировка для заполнения пропусков дохода: по типу дохода и образованию.  
</span></div>

In [1128]:
# Проверка результатов замен пропусков и нулевых значений
data.describe()

,children,days_employed,dob_years,education_id,family_status_id,debt,total_income
count,21525.00,21521.00,21525.00,21525.00,21525.00,21525.00,21525.00
mean,0.48,63509.94,43.50,0.82,0.97,0.08,165375.89
std,0.76,141126.94,12.23,0.55,1.42,0.27,98164.25
min,0.00,-18388.95,19.00,0.00,0.00,0.00,20667.26
25%,0.00,-2573.91,34.00,1.00,0.00,0.00,107714.24
50%,0.00,-1283.69,43.00,1.00,0.00,0.00,143111.63
75%,1.00,-316.24,53.00,1.00,1.00,0.00,198105.57
max,5.00,401755.40,75.00,4.00,4.00,1.00,2265604.03


**Вывод**

<br>Пропуски в столбцах <span style='color:darkblue'>**days_employed**</span> и <span style='color:darkblue'>**total_income**</span> совпадают - в строках где пропущен стаж, также пропущено значение дохода заемщика. Скорее всего часть заемщиков не может подтвердить официально стаж и соответственно доход, т.е. имеют неофицальные источники дохода. В столбце стажа также имеются <span style='color:darkblue'>**аномально высокие (все - плюсовые)**</span> данные о стаже в годах, например <span style='color:darkblue'>**1100 лет**</span>. Такие данные оставлены без изменений, необходима дополнительная информация от поставщика данных о причинах или логике их появления.

<br><span style='color:darkblue'>**Плюсовые**</span> аномально высокие данные по стажу <span style='color:darkblue'>**только у пенсионеров и безработных**.</span>. Необходим запрос поставщику данных о причинах аномально высоких плюсовых значениях у этих двух групп и минусовых но логичных значений у всех остальных. Пропуски в столбце данных стажа, составляют <span style='color:darkblue'>**10%**</span>, заменены на <span style='color:darkblue'>**медианные**</span> значения группировки типа дохода и возраста, по логике наличия взаимосвязи.

<br>Пропуски дохода в столбце <span style='color:darkblue'>**total_income**</span> - <span style='color:darkblue'>**10%**</span>, заполнены по <span style='color:darkblue'>**медианному**</span> значению соответвстующего <span style='color:darkblue'>группировке **типа занятости и образования**</span> столбца <span style='color:darkblue'>**uncome_type**</span>, т.к. в столбце с тиапми дохода присутствует данные по каждой строк без пропусков и логически взаимосвязаны с доходом.</span>

<br><span style='color:darkblue'>**Минусовые**</span> значения количества детей в столбце <span style='color:darkblue'>**children**</span> приведены <span style='color:darkblue'>**к плюсовым**</span>, т.к. минусовое значение в этом столбце явно ошибочное. Данные (76 строк, 0.35%) с количеством детей - 20, приведены к значению 2, т.к. 20 детей -аномально высокое значение в выборке, явнвя ошибка заполнения данных.</span>

<br><span style='color:darkblue'>**Нулевые**</span> значения в столбце возраста <span style='color:darkblue'>**dob_years**</span> заполнены на <span style='color:darkblue'>**медианный возраст**</span> по соответсвующему <span style='color:darkblue'>**типу доходов**</span> в столбце <span style='color:darkblue'>**income_type**</span>, т.к. столбец типа дохода не имеет пропусков и логически взаимосвязан с возрастом клиента. Замененные значения стажа составили <span style='color:darkblue'>**0.26%**</span>, такая замена не приведет к искажению данных.

<br><span style='color:darkblue'>**Третье значение пола**</span> в столбце <span style='color:darkblue'>**gender**</span>, в одной строк со значением <span style='color:darkblue'>**XMA**</span> заменено на значение <span style='color:darkblue'>**F**</span> как <span style='color:darkblue'>**наиболее частотное**</span>.

### Замена типа данных

In [1129]:
# Проверка текущего типа данных
data.dtypes

children              int64
days_employed       float64
dob_years           float64
education            object
education_id          int64
family_status        object
family_status_id      int64
gender               object
income_type          object
debt                  int64
total_income        float64
purpose              object
dtype: object

In [1130]:
# Замена типа float на int в данных столбца возраста
data['dob_years'] = data['dob_years'].astype('int64')

In [1131]:
# Проверка текущего типа данных
data.dtypes

children              int64
days_employed       float64
dob_years             int64
education            object
education_id          int64
family_status        object
family_status_id      int64
gender               object
income_type          object
debt                  int64
total_income        float64
purpose              object
dtype: object

**Вывод**

Был изменен <span style='color:darkblue'>**тип данных**</span> в столбце возраста <span style='color:darkblue'>**dob_years**</span>, который уже <span style='color:darkblue'>**подвергался обработке**</span> при заполнении нулевых значений, с <span style='color:darkblue'>**дробного на целочисленный**</span> методом <span style='color:darkblue'>**astype()**</span>

### Обработка дубликатов

In [1132]:
# Просмотр категориальных признаков, определение наличия повторов
print(data['education'].unique())
print(data['family_status'].unique())
print(data['income_type'].unique())
print(data['purpose'].unique())

['высшее' 'среднее' 'Среднее' 'СРЕДНЕЕ' 'ВЫСШЕЕ' 'неоконченное высшее'
 'начальное' 'Высшее' 'НЕОКОНЧЕННОЕ ВЫСШЕЕ' 'Неоконченное высшее'
 'НАЧАЛЬНОЕ' 'Начальное' 'Ученая степень' 'УЧЕНАЯ СТЕПЕНЬ'
 'ученая степень']
['женат / замужем' 'гражданский брак' 'вдовец / вдова' 'в разводе'
 'Не женат / не замужем']
['сотрудник' 'пенсионер' 'компаньон' 'госслужащий' 'безработный'
 'предприниматель' 'студент' 'в декрете']
['покупка жилья' 'приобретение автомобиля' 'дополнительное образование'
 'сыграть свадьбу' 'операции с жильем' 'образование'
 'на проведение свадьбы' 'покупка жилья для семьи' 'покупка недвижимости'
 'покупка коммерческой недвижимости' 'покупка жилой недвижимости'
 'строительство собственной недвижимости' 'недвижимость'
 'строительство недвижимости' 'на покупку подержанного автомобиля'
 'на покупку своего автомобиля' 'операции с коммерческой недвижимостью'
 'строительство жилой недвижимости' 'жилье'
 'операции со своей недвижимостью' 'автомобили' 'заняться образованием'
 'сделка

<br><span style='color:blue'>В столбце **education** - разный регистр одинаковых значений</span>
<br><span style='color:blue'>В столбце **family_status** - дубликатов значений нет</span>
<br><span style='color:blue'>В столбце **income_type** - дублткатов значений нет</span>
<br><span style='color:blue'>В столбце **purpose** - есть дубликаты по смыслу, столбец будет леммизирован</span>

In [1133]:
# Приведение столбца education к нижнему регистру
data['education'] = data['education'].str.lower()
data['education'].value_counts().sum()

21525

In [1134]:
# Вывод количества и доли дубликатов
print('колличество дублированных строк:', data.duplicated().sum())
print('Доля дубликатов в общем Dataframe: {:.2%}'.format(data.duplicated().sum() / len(data)))

колличество дублированных строк: 54
Доля дубликатов в общем Dataframe: 0.25%


In [1135]:
# Визуальная оценка дубликатов
data[data.duplicated(keep=False)].sort_values(by=['dob_years'])

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
19321,0,-740.20,23,среднее,1,Не женат / не замужем,4,F,сотрудник,0,136818.12,сделка с подержанным автомобилем
15892,0,-740.20,23,среднее,1,Не женат / не замужем,4,F,сотрудник,0,136818.12,сделка с подержанным автомобилем
18328,0,-1272.24,29,высшее,0,женат / замужем,0,M,сотрудник,0,166164.08,покупка жилой недвижимости
3452,0,-1272.24,29,высшее,0,женат / замужем,0,M,сотрудник,0,166164.08,покупка жилой недвижимости
8629,1,-1361.78,30,высшее,0,женат / замужем,0,F,сотрудник,0,166164.08,покупка коммерческой недвижимости
...,...,...,...,...,...,...,...,...,...,...,...,...
13639,0,366617.00,64,среднее,1,женат / замужем,0,F,пенсионер,0,114483.37,автомобиль
3609,0,366617.00,64,среднее,1,женат / замужем,0,F,пенсионер,0,114483.37,жилье
12389,0,366617.00,64,среднее,1,женат / замужем,0,F,пенсионер,0,114483.37,дополнительное образование
5865,0,371258.56,66,среднее,1,вдовец / вдова,2,F,пенсионер,0,114483.37,операции со своей недвижимостью


In [1136]:
# Удаление дубликатов
data = data.drop_duplicates().reset_index(drop = True)
print('Число дубликатов в массиве данных:', data.duplicated().sum()) # Проверка

Число дубликатов в массиве данных: 0


**Вывод**

<br><span style='color:blue'> **54** полных дубликатов удалено, что составило **0.25%** от длинны Dataframe. Рекомендация поставщику данных - ввести в их порядок формирования уникальный id каждому заемщику.</span>

### Лемматизация

In [1137]:
# Импрорт библиотек pymystem3 и collections
from pymystem3 import Mystem
from collections import Counter
m = Mystem()

In [1138]:
# Подсчет количества каждой цели кредита
data['purpose'].value_counts()

свадьба                                   793
на проведение свадьбы                     773
сыграть свадьбу                           769
операции с недвижимостью                  675
покупка коммерческой недвижимости         662
покупка жилья для сдачи                   652
операции с жильем                         652
операции с коммерческой недвижимостью     650
жилье                                     646
покупка жилья                             646
покупка жилья для семьи                   638
строительство собственной недвижимости    635
недвижимость                              633
операции со своей недвижимостью           627
строительство жилой недвижимости          625
покупка недвижимости                      621
покупка своего жилья                      620
строительство недвижимости                619
ремонт жилью                              607
покупка жилой недвижимости                606
на покупку своего автомобиля              505
заняться высшим образованием      

<br><span style='color:blue'> После оценки списка выделено **шесть основных целей** получения кредита по словам ключевым словам для создания списка категорий на основании которго после выделения лемм будут категоризированы цели</span>

In [1139]:
# Создание списка ключевых слов 
purpose_categories = ['сдача', 'коммерческий', 'жилье','образование', 'свадьба', 'недвижимость', 'автомобиль']

<div class="alert alert-warning" style= "background-color: #d3e5f0" style="border-radius: 15px"><span style='color:blue'>Ниже цели перегруппированы в обновленный столбец purposes_cat: в недвижимость, свадьба, автомобиль и образование. В цель недвижимость вошли "сдача", "жилье", "коммерческий" как близкие по смыслу. 
</span></div>

In [1140]:
# Проведение лемматизации, замена полученных лемм на ключевое слово списка 
def lemmatize(text):
    lemma = m.lemmatize(text)
    for word in purpose_categories:
        if word in lemma:
            lemma = word
    return lemma

data['purpose_groups'] = data['purpose'].apply(lemmatize)        
data.head() # Оценка результата

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose,purpose_groups
0,1,-8437.67,42,высшее,0,женат / замужем,0,F,сотрудник,0,253875.64,покупка жилья,жилье
1,1,-4024.80,36,среднее,1,женат / замужем,0,F,сотрудник,0,112080.01,приобретение автомобиля,автомобиль
2,0,-5623.42,33,среднее,1,женат / замужем,0,M,сотрудник,0,145885.95,покупка жилья,жилье
3,3,-4124.75,32,среднее,1,женат / замужем,0,M,сотрудник,0,267628.55,дополнительное образование,образование
4,0,340266.07,53,среднее,1,гражданский брак,1,F,пенсионер,0,158616.08,сыграть свадьбу,свадьба


In [1141]:
# Подсчет значений всех групп целей кредита
data['purpose_groups'].value_counts()

недвижимость    5041
автомобиль      4308
образование     4014
жилье           3809
свадьба         2335
коммерческий    1312
сдача            652
Name: purpose_groups, dtype: int64

In [1142]:
# Замена цели "жилье" на "недвижимость", "сдача" на инвестиции в недвижимость
data.loc[(data['purpose_groups'] == 'жилье')
| (data['purpose_groups'] == 'коммерческий') | (data['purpose_groups'] == 'сдача'), 'purpose_groups'] = 'недвижимость'
data['purpose_groups'].value_counts()

недвижимость    10814
автомобиль       4308
образование      4014
свадьба          2335
Name: purpose_groups, dtype: int64

<br><span style='color:blue'>Цель **"жилье"**, **коммерческий** и **сдача** вошла в цель **"недвижимость"** по равнозначному смыслу.</span>

**Вывод**

Логически категоризированы цели по <span style='color:darkblue'>**ключевым словам**</span>, по типу <span style='color:darkblue'>**"свадьба"**</span> для целей содержащих <span style='color:darkblue'>**сыграть свадьбу"**</span> и <span style='color:darkblue'>**"на проведение свадьбы"**</span> для упрощения дальнейшего анализа.

Обеденены логически цели содержащие ключевое слово <span style='color:darkblue'>**"жилье"</span>, **"коммерческий"**, **"сдача"** и <span style='color:darkblue'>**"недвижимость"**</span> в общую категорию <span style='color:darkblue'>**"недвижимость"**</span>.

<span style='color:darkblue'>Лемматизированные цели добавлены в столбец purposes_cat</span>.

### Категоризация данных

#### Категоризация столбца children

In [1143]:
# Добавление столбца с отметкой о наличии или отсутствии детей 
def children_mark(children):
    if children > 0: return 'есть дети'
    else: return 'нет детей'
data['children_status'] = data['children'].apply(children_mark)
data['children_status'].value_counts()

нет детей    14107
есть дети     7364
Name: children_status, dtype: int64

Cтолбец с количеством детей children отмаркирован по наличию детей

#### Категоризация столбца доходы total_income

In [1144]:
# Получение значений квантилей из описания столбца
income_stat_data = data['total_income'].describe()
income_stat_data

count     21471.00
mean     165449.11
std       98271.01
min       20667.26
25%      107520.53
50%      143206.18
75%      198251.97
max     2265604.03
Name: total_income, dtype: float64

In [1145]:
# Определение категории доходов по квантилям
def income_group_mark(income):
    if income <= 100000: return 'менее 100000 р'
    elif 100000 < income <= 150000: return 'от 100000 р до 150000 р'
    elif 150000 < income <= 200000: return 'от 150000 р до 200000 р'
    else: return 'более 200000 р'

# Применим функцию к столбцу доходов
data['income_cat'] = data['total_income'].apply(income_group_mark)
data.head()

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose,purpose_groups,children_status,income_cat
0,1,-8437.67,42,высшее,0,женат / замужем,0,F,сотрудник,0,253875.64,покупка жилья,недвижимость,есть дети,более 200000 р
1,1,-4024.80,36,среднее,1,женат / замужем,0,F,сотрудник,0,112080.01,приобретение автомобиля,автомобиль,есть дети,от 100000 р до 150000 р
2,0,-5623.42,33,среднее,1,женат / замужем,0,M,сотрудник,0,145885.95,покупка жилья,недвижимость,нет детей,от 100000 р до 150000 р
3,3,-4124.75,32,среднее,1,женат / замужем,0,M,сотрудник,0,267628.55,дополнительное образование,образование,есть дети,более 200000 р
4,0,340266.07,53,среднее,1,гражданский брак,1,F,пенсионер,0,158616.08,сыграть свадьбу,свадьба,нет детей,от 150000 р до 200000 р


<span style='color:blue'>Добавлен новый столбец **income_cat** с категориями на основе квантилей для столбца доход **total_income**. Поскольку использование значений квантилей в их точном числовом значении затрудняет визуально анализ и восприятие, группы прведены к близлежащим круглым значениям в рублях. Например группа доходов **от 100000 р до 150000 р**, вместо **от 107623 р до 142594 р**</span>

<span style='color:blue'> Итоговые категории дохода:
    
 <br><span style='color:blue'> **менее 100000 р | от 100000 р до 150000 р | от 100000 р до 150000 р | более 200000 р** </span>
 </span>

#### Категоризация возраста заемщиков dob_years

In [1146]:
# Получение значений квантилей из описания столбца
age_stat_data = data['dob_years'].describe()
age_stat_data

count   21471.00
mean       43.48
std        12.23
min        19.00
25%        33.50
50%        43.00
75%        53.00
max        75.00
Name: dob_years, dtype: float64

In [1147]:
# Определение категории возраста по квантилям
def age_group_mark(age):
    if age <= age_stat_data[4]: return 'младше 33'
    elif age_stat_data[4] < age <= age_stat_data[5]: return 'от 33 до 42'
    elif age_stat_data[5] < age <= age_stat_data[6]: return 'от 42 до 53'
    else: return 'старше 53'
# Применение функции к столбцу возраста
data['age_cat'] = data['dob_years'].apply(age_group_mark)
data.head()

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose,purpose_groups,children_status,income_cat,age_cat
0,1,-8437.67,42,высшее,0,женат / замужем,0,F,сотрудник,0,253875.64,покупка жилья,недвижимость,есть дети,более 200000 р,от 33 до 42
1,1,-4024.80,36,среднее,1,женат / замужем,0,F,сотрудник,0,112080.01,приобретение автомобиля,автомобиль,есть дети,от 100000 р до 150000 р,от 33 до 42
2,0,-5623.42,33,среднее,1,женат / замужем,0,M,сотрудник,0,145885.95,покупка жилья,недвижимость,нет детей,от 100000 р до 150000 р,младше 33
3,3,-4124.75,32,среднее,1,женат / замужем,0,M,сотрудник,0,267628.55,дополнительное образование,образование,есть дети,более 200000 р,младше 33
4,0,340266.07,53,среднее,1,гражданский брак,1,F,пенсионер,0,158616.08,сыграть свадьбу,свадьба,нет детей,от 150000 р до 200000 р,от 42 до 53


In [1148]:
# Группировка  пола для последующего анализа
age_gender_probability = data.groupby('gender')['debt'].agg(
    total_clients='count',debitors='sum',debt_probability_pct=lambda x:
    (x.sum() / x.count())*100).sort_values(by='debt_probability_pct',ascending=False)
age_gender_probability

,total_clients,debitors,debt_probability_pct
gender,,,
M,7281,747,10.26
F,14190,994,7.00


In [1149]:
# Группировка возраста и пола для последующего анализа
age_gender_probability = data.groupby(['age_cat','gender'])['debt'].agg(
    total_clients='count',debitors='sum',debt_probability_pct=lambda x:
    (x.sum() / x.count())*100).sort_values(by='debt_probability_pct',ascending=False)
age_gender_probability

total_clients  debitors  debt_probability_pct
age_cat     gender                                               
младше 33   M                2235       268                 11.99
            F                3133       319                 10.18
от 33 до 42 M                2181       218                 10.00
от 42 до 53 M                1574       152                  9.66
старше 53   M                1291       109                  8.44
от 33 до 42 F                3697       286                  7.74
от 42 до 53 F                3367       203                  6.03
старше 53   F                3993       186                  4.66

<span style='color:blue'>Добавлен новый столбец **age_cat** с категориями на основе квантилей для столбца возраст **dob_years**. Также использованы квартили для равномерной группировки по всему распределению. Группировка возраста и пола необходима для финального вывода.</span>

**Вывод**

Были категоризированы данные по столбцу количества детей <span style='color:darkblue'>**children** в столбец <span style='color:darkblue'>**children status**, в виде значений  "дети есть" или "детей нет". Данная категория необходима для ответа на вопрос: <span style='color:darkblue'>*влияет ли наличие детей на платежеспособность*.</span>

Были категоризированы доходы по круглым значениям близким к квантилям столбца <span style='color:darkblue'>**total_income в** столбец <span style='color:darkblue'>**income_cat** для анализа доходов и ответа на вопрос: <span style='color:darkblue'>*есть ли зависимость дохода и погащением кредита в срок.*</span>

Был категоризирован данные по столбцу возраста клиентов <span style='color:darkblue'>**dob_years** в столбец <span style='color:darkblue'>**age_cat** по квантилям для упрощения анализа.</span>

Ранее были лемматизированы и категоризированы цели кредита в обновленный столбец <span style='color:darkblue'>**purposes**</span> в виде пяти вариантов целей для ответа на вопрос: <span style='color:darkblue'>*как разные цели кредита влияют на его возврат в срок.*</span>

Ответ на вопрос: <span style='color:darkblue'>*есть ли зависимость между семейным положением и возвратом кредита в срок,*</span> будет получен после анализа текущих значений столбца семейное положение <span style='color:darkblue'>**family_status**</span>

**Квантили** послужили основой для категоризации доходов по близким к ним округленным значениям и границами групп возраста. Выбраны для анализа всей широты распределения значений.</span>

## Шаг 3. Ответьте на вопросы

- Есть ли зависимость между наличием детей и возвратом кредита в срок?

In [1150]:
# Группировка просрочки по наличию детей
children_status_probability = data.groupby('children_status')['debt'].agg(
    total_clients='count',debitors='sum',debt_probability_pct=lambda x: (x.sum() / x.count())*100)
children_status_probability

,total_clients,debitors,debt_probability_pct
children_status,,,
есть дети,7364,678,9.21
нет детей,14107,1063,7.54


<span style='color:blue'>**Зависимость** между наличием детей и возвратом кредита в срок **существует**. Вероятность просрочки выше у заемщиков с детьми. Дети - большая статья расходов. Ниже рассмотрим гипотезу: чем больше детей тем более вероятна просрочка.</span>

In [1151]:
# Группировка просрочки по количеству детей
qty_children_probability = data.groupby('children')['debt'].agg(
    total_clients='count',debitors='sum',debt_probability_pct=lambda x:
    (x.sum() / x.count())*100).sort_values(by='debt_probability_pct',ascending=False)
qty_children_probability

,total_clients,debitors,debt_probability_pct
children,,,
4,41,4,9.76
2,2128,202,9.49
1,4856,445,9.16
3,330,27,8.18
0,14107,1063,7.54
5,9,0,0.00


<span style='color:blue'>**Количество детей** ожидаемо **влияет** на вероятность просрочки, чем больше детей тем она выше Однако вероятные исключения, заемщики с **5** детьми не допускли просрочек, хотя число таких заемщиков не велико. Заемщики с **3** детьми оказались отетственнее чем заемщики с двумя. Ниже рассмотрим количество детей в связке с доходом, для выяснения этой странности, ведь дети требуют серьезных расходов.</span>

In [1152]:
# Группировка просрочки по количеству детей и уровню дохода
qty_children_income_probability = data.groupby(['children','income_cat'])['debt'].agg(
    total_clients='count',debitors='sum',debt_probability_pct=lambda x:
    (x.sum() / x.count())*100).sort_values(by='debt_probability_pct',ascending=False)
qty_children_income_probability

total_clients  debitors  \
children income_cat                                         
4        менее 100000 р                       5         1   
         более 200000 р                       7         1   
3        менее 100000 р                      59         8   
2        от 100000 р до 150000 р            675        73   
1        от 100000 р до 150000 р           1598       161   
2        от 150000 р до 200000 р            460        46   
3        от 150000 р до 200000 р             73         7   
1        менее 100000 р                     898        86   
         от 150000 р до 200000 р           1138       108   
2        менее 100000 р                     448        38   
         более 200000 р                     545        45   
0        от 100000 р до 150000 р           4635       374   
         от 150000 р до 200000 р           3040       242   
3        от 100000 р до 150000 р            106         8   
1        более 200000 р                    1222        90   
0        менее 100000 р                    3053       221   
4        от 150000 р до 200000 р             14         1   
0        более 200000 р                    3379       226   
4        от 100000 р до 150000 р             15         1   
3        более 200000 р                      92         4   
5        более 200000 р                       4         0   
         менее 100000 р                       2         0   
         от 100000 р до 150000 р              2         0   
         от 150000 р до 200000 р              1         0   

                                  debt_probability_pct  
children income_cat                                     
4        менее 100000 р                          20.00  
         более 200000 р                          14.29  
3        менее 100000 р                          13.56  
2        от 100000 р до 150000 р                 10.81  
1        от 100000 р до 150000 р                 10.08  
2        от 150000 р до 200000 р                 10.00  
3        от 150000 р до 200000 р                  9.59  
1        менее 100000 р                           9.58  
         от 150000 р до 200000 р                  9.49  
2        менее 100000 р                           8.48  
         более 200000 р                           8.26  
0        от 100000 р до 150000 р                  8.07  
         от 150000 р до 200000 р                  7.96  
3        от 100000 р до 150000 р                  7.55  
1        более 200000 р                           7.36  
0        менее 100000 р                           7.24  
4        от 150000 р до 200000 р                  7.14  
0        более 200000 р                           6.69  
4        от 100000 р до 150000 р                  6.67  
3        более 200000 р                           4.35  
5        более 200000 р                           0.00  
         менее 100000 р                           0.00  
         от 100000 р до 150000 р                  0.00  
         от 150000 р до 200000 р                  0.00

<span style='color:blue'>Заемщики с **5** детьми не допускают просрочек при любом уровне доходов, возможно меры поддержки государства играют роль.Среди заемщиков с **3** детьми действительно меньше просрочек чему у заемщиков с **2** детьми. в РФ заемщики с тремя детьми уже считаются многодетными семьями и получают дополнительную поддержку от федеральных властей и часто - от местных, что может помогать им исполнять обязательства в срок. Заемщики с **4** детьми достаточно дисциплинированы за исключением их самых небогатых и наоборот самых обеспеченных представителей Заемщики с **5** детьми не допускают просрочек при любом уровне доходов, возможно меры поддержки государства играют роль.</span>

**Вывод**

Зависимость между наличием детей и исполнением обязательств в срок присутствует. Вероятность просрочки у заемщиков с детьми выше - <span style='color:darkblue'>**9.21%**</span>  против <span style='color:darkblue'>**7.54%**</span>. С увеличением количества детей у заемщика вероятность просрочки увеличивается, однако при наличии <span style='color:darkblue'>**3 и более детей**</span> возможны исключения. Вероятно играет роль господдержка.

- Есть ли зависимость между семейным положением и возвратом кредита в срок?

In [1153]:
# Группировка просрочки по семейному положению заемщиков
familiy_stat_probability = data.groupby('family_status')['debt'].agg(
    total_clients='count',debitors='sum',debt_probability_pct=lambda x:
    (x.sum() / x.count())*100).sort_values(by='debt_probability_pct',ascending=False)
familiy_stat_probability

,total_clients,debitors,debt_probability_pct
family_status,,,
Не женат / не замужем,2810,274,9.75
гражданский брак,4163,388,9.32
женат / замужем,12344,931,7.54
в разводе,1195,85,7.11
вдовец / вдова,959,63,6.57


<span style='color:blue'>Наиболее ответственны в погашении кредитов одинокие заемщики, однако бывшие в браке. Официальный брак также влияет на снижение просрочек.</span>

In [1154]:
# Группировка просрочки по семейному положению заемщиков и наличию детей
family_stat_children_probability = data.groupby(['family_status','children_status'])['debt'].agg(
    total_clients='count',debitors='sum',debt_probability_pct=lambda x:
    (x.sum() / x.count())*100).sort_values(by='debt_probability_pct',ascending=False)
family_stat_children_probability

total_clients  debitors  \
family_status         children_status                            
Не женат / не замужем есть дети                  548        64   
гражданский брак      есть дети                 1422       159   
Не женат / не замужем нет детей                 2262       210   
вдовец / вдова        есть дети                  112        10   
женат / замужем       есть дети                 4871       415   
гражданский брак      нет детей                 2741       229   
в разводе             есть дети                  411        30   
                      нет детей                  784        55   
женат / замужем       нет детей                 7473       516   
вдовец / вдова        нет детей                  847        53   

                                       debt_probability_pct  
family_status         children_status                        
Не женат / не замужем есть дети                       11.68  
гражданский брак      есть дети                       11.18  
Не женат / не замужем нет детей                        9.28  
вдовец / вдова        есть дети                        8.93  
женат / замужем       есть дети                        8.52  
гражданский брак      нет детей                        8.35  
в разводе             есть дети                        7.30  
                      нет детей                        7.02  
женат / замужем       нет детей                        6.90  
вдовец / вдова        нет детей                        6.26

<span style='color:blue'>Подтверждается ранее сделанный вывод - наличие детей увеличивает вероятность просрочки вне зависимости от семейного статуса. Заемщики состоящие в браке более дисциплинированы как и заемщики бывшие в браке, вне зависимости от наличия детей.</span>

**Вывод**

Зависимость между семейным положением и возвратом кредита в срок присутствует. Официальный брак снижает риск невозврата в срок с вероятностью <span style='color:darkblue'>**7.55%**</span> против гражданского брака и одиноких заемщиков <span style='color:darkblue'>**9.35%**</span> и <span style='color:darkblue'>**9.75%**</span> соответственно. Олинокие заемщики бывшие в браке - самые дисциплиннированные заемщики, их вероятность просрочки - <span style='color:darkblue'>**менее 7%**.</span>

- Есть ли зависимость между уровнем дохода и возвратом кредита в срок?

In [1155]:
# Группировка просрочки по доходу заемщиков
income_probability = data.groupby('income_cat')['debt'].agg(
    total_clients='count',debitors='sum',debt_probability_pct=lambda x:
    (x.sum() / x.count())*100).sort_values(by='debt_probability_pct',ascending=False)
income_probability

,total_clients,debitors,debt_probability_pct
income_cat,,,
от 100000 р до 150000 р,7031,617,8.78
от 150000 р до 200000 р,4726,404,8.55
менее 100000 р,4465,354,7.93
более 200000 р,5249,366,6.97


<span style='color:blue'>Вероятность просрочки больше - **у среднего-минус класса по доходам**, самые небогатые и самые обеспеченные - самые ответственные заемщики. Тут необходимо рассмотреть взаимосвязь между возникновением просрочк и связки дохода с образованием заемщика, т.к. наличие образования в целом важно для зарабатывания денег.</span>

In [1156]:
# Группировка просрочки доходу и образованию заемщиков
income_education_probability = data.groupby(['income_cat','education'])['debt'].agg(
    total_clients='count',debitors='sum',debt_probability_pct=lambda x:
    (x.sum() / x.count())*100).sort_values(by='debt_probability_pct',ascending=False)
income_education_probability

total_clients  debitors  \
income_cat              education                                      
от 150000 р до 200000 р начальное                       44         6   
от 100000 р до 150000 р начальное                      103        14   
более 200000 р          неоконченное высшее            219        27   
                        начальное                       41         5   
от 150000 р до 200000 р среднее                       3125       298   
от 100000 р до 150000 р среднее                       5484       519   
                        неоконченное высшее            229        21   
менее 100000 р          среднее                       3648       304   
более 200000 р          среднее                       2931       243   
менее 100000 р          неоконченное высшее            115         8   
от 150000 р до 200000 р неоконченное высшее            181        12   
                        высшее                        1375        88   
менее 100000 р          начальное                       94         6   
                        высшее                         607        36   
от 100000 р до 150000 р высшее                        1213        63   
более 200000 р          высшее                        2056        91   
от 100000 р до 150000 р ученая степень                   2         0   
менее 100000 р          ученая степень                   1         0   
более 200000 р          ученая степень                   2         0   
от 150000 р до 200000 р ученая степень                   1         0   

                                             debt_probability_pct  
income_cat              education                                  
от 150000 р до 200000 р начальное                           13.64  
от 100000 р до 150000 р начальное                           13.59  
более 200000 р          неоконченное высшее                 12.33  
                        начальное                           12.20  
от 150000 р до 200000 р среднее                              9.54  
от 100000 р до 150000 р среднее                              9.46  
                        неоконченное высшее                  9.17  
менее 100000 р          среднее                              8.33  
более 200000 р          среднее                              8.29  
менее 100000 р          неоконченное высшее                  6.96  
от 150000 р до 200000 р неоконченное высшее                  6.63  
                        высшее                               6.40  
менее 100000 р          начальное                            6.38  
                        высшее                               5.93  
от 100000 р до 150000 р высшее                               5.19  
более 200000 р          высшее                               4.43  
от 100000 р до 150000 р ученая степень                       0.00  
менее 100000 р          ученая степень                       0.00  
более 200000 р          ученая степень                       0.00  
от 150000 р до 200000 р ученая степень                       0.00

**Вывод**

Зависимость между уровнем дохода и возвратом кредита в срок присутствует. У заемщиков с уровнем дохода "средний минус" <span style='color:darkblue'>**от 100000 до 150000 р**</span>. - самая высокая вероятность возникновения просрочки - <span style='color:darkblue'>**8.78%**</span> против <span style='color:darkblue'>**6.97%**</span> у самых обеспеченных заемшиков. Стоить заметить, что наименее обеспеченные заемщики, также дисциплиннированы с вероятностью просрочки <span style='color:darkblue'>**7.93%**</span>
Образование через доходы также влияет на вероятность просрочки, <span style='color:darkblue'>**чем выше его уровень**</span> - тем она менее вероятна, вне зависимости от доходов, за исключением самых небогатых заемщиков с начальным образованием и доходом менее <span style='color:darkblue'>**100000**</span> р.<span style='color:darkblue'>(вероятность просрочки **6.38%**)</span>, они наиболии финансово дисциплинированы, после заемщиков с вышим образованием и учеными степенями.
Если заемщиков с высшим образованием и ученых вероятно отличает более высокий уровень финансовой грамотности и способность обеспечить себе высокий доход, то малообеспеченные и малообразованные заемщики возможно более способны ограничить себя в расходах, либо берут небольшие суммы по своему доходу и возможностям. Другая вероятность для этой категории заемщиков - высокий отсев на этапе одобрения кредитов.

- Как разные цели кредита влияют на его возврат в срок?

In [1157]:
# Группировка просрочки по целям кредита
purpose_probability = data.groupby('purpose_groups')['debt'].agg(
    total_clients='count',debitors='sum',debt_probability_pct=lambda x:
    (x.sum() / x.count())*100).sort_values(by='debt_probability_pct',ascending=False)
purpose_probability

,total_clients,debitors,debt_probability_pct
purpose_groups,,,
автомобиль,4308,403,9.35
образование,4014,370,9.22
свадьба,2335,186,7.97
недвижимость,10814,782,7.23


**Вывод**

Цели получения кредита влияют на возникновение просрочек. Самая высокая вероятность возникновения просрочки у заемщиков покупающих в кредит автомобиль, несмотря на оформление залога - <span style='color:darkblue'>**9.35%**</span>, как движимое имущество автомобиль подвержен высоким рискам утраты или повреждения, несмотря на страхование. Для него характерен быстрый износ и следовательно обесценение как предмета залога, он требует постоянных расходов на содержание.

Высокую вероятность просрочки по образовательному кредиту - <span style='color:darkblue'>**9.22%**</span> можно объяснить в том числе проблемами с выбором учебного заведения, программы и направления обучения, переоценкой своих возможностей и как следстве - невозможности достичь дохода, позволяющего обслуживать такой кредит.

Приобретение недвижимости как цель, несет в себе самую низкую вероятность просрочки <span style='color:darkblue'>**7.23%**</span> Недвижимость - хороший залог, почти не подвержена обесценению, для большинства заемщиков - одно из главных приобретений в жизни, базовая потребность иметь крышу над головой, что в итоге стимулирует платежную дисциплину.

## Шаг 4. Общий вывод

На все поставленные вопросы даны утвердительные ответы, описаны выводы и возможные взаимосвязи для них. Выявленны зависимости появления просрочки кредита от наличия детей, семейного положения, уровня доходов заемщика, а также влияние на просрочку целей получения кредита.

<br>Возможно составить портрет клиента с наименьшей вероятностью появления просрочки по кредиту исходя из полученных данных:

<div class="alert alert-warning" style= "background-color: #d3e5f0" style="border-radius: 15px"><br>Это женщина от 53 лет и старше, без детей, и вдова с доходом свыше 200000 р, имеющая  ученую степень, берущая кредит на приобретение недвижимости.</div>

<br>Возможно также составить портрет клиента с наибольшей вероятностью появления просрочки:


<div class="alert alert-warning" style= "background-color: #d3e5f0" style="border-radius: 15px"><br>Это мужчина младше 33 лет, с 4 детьми, не женат, с доходом 100000 - 150000 р, начальное образование, берущий кредит на приобретение автомобиля.</div>

    
| Признак | Лучший | (% невозврата) | Худший | (% невозврата) |
| :--- | :--- | ---: | :--- | ---: |
| Пол и возраст        | Женщина 53+                  |  4.66 | Мужчина 33-                   | 11.99 |
| Дети                 | нет детей                    |  7.54 | четверо                       | 9.76  |  
| Семейный статус      | вдова                        |  6.26 | не женат                      | 9.75  |
| Доход и образование  | более 200 т.р ученая степень |  0.00 | 100 тр -150 т.р. начальное    | 13.64 |
| Цель кредита         | недвижимость                 |  7.23 | автомобиль                    | 9.35  |